In [4]:
# ------------------ Config + Imports ------------------

import os
import numpy as np
import librosa
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

IMG_SIZE = (128, 128)
DATASET_PATH = "D:/Downloads/archive/Emotions"
BATCH_SIZE = 32
EPOCHS = 50

# ------------------ Feature Extraction ------------------

def extract_mel_spectrogram(file_path, img_size=(128, 128), augment=False):
    y, sr = librosa.load(file_path, sr=22050)

    if augment:
        if np.random.rand() < 0.5:
            y = y + 0.005 * np.random.randn(len(y))  # Zgomot alb
        if np.random.rand() < 0.5:
            y = librosa.effects.pitch_shift(y, sr=sr, n_steps=np.random.choice([-1, 1]))

    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=img_size[0])
    S_DB = librosa.power_to_db(S, ref=np.max)
    S_DB = librosa.util.fix_length(S_DB, size=img_size[1], axis=1)
    return S_DB

# ------------------ Dataset Loader ------------------

def load_dataset():
    X, y = [], []

    for emotion_folder in os.listdir(DATASET_PATH):
        folder_path = os.path.join(DATASET_PATH, emotion_folder)
        if not os.path.isdir(folder_path):
            continue

        for file in tqdm(os.listdir(folder_path), desc=emotion_folder):
            if not file.endswith(".wav"):
                continue
            try:
                file_path = os.path.join(folder_path, file)
                spec = extract_mel_spectrogram(file_path, IMG_SIZE, augment=True)
                X.append(spec)
                y.append(emotion_folder)
            except Exception as e:
                print(f"Error with file {file}: {e}")
                continue

    X = np.array(X)[..., np.newaxis]  # (samples, H, W, 1)
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    y_cat = to_categorical(y_encoded)
    return X, y_cat, le

# ------------------ CNN Model ------------------

def build_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.3),

        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.3),

        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.3),

        GlobalAveragePooling2D(),
        Dense(256, activation='relu'),
        Dropout(0.4),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=3e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

 #------------------ Training Pipeline ------------------

# Incarca datele
X, y, label_encoder = load_dataset()

# Impartire train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42)

# Construieste modelul
model = build_cnn_model(input_shape=X.shape[1:], num_classes=y.shape[1])
model.summary()

# Callback-uri
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
]

# Antrenam modelul
model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks
)

# Evaluam modelul
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.2f}")

# Salvam modelul in format compatibil TensorFlow
model.save("emotion_cnn_audio_only.keras", save_format="tf")


Suprised: 100%|██████████| 592/592 [00:28<00:00, 20.66it/s]
d:\proiect_ai\EmotionOfficialCommunications\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 128, 128, 64)   │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64, 64, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32, 32, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 439,047 (1.67 MB)

 Trainable params: 438,151 (1.67 MB)

 Non-trainable params: 896 (3.50 KB)

Epoch 1/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 619s 2s/step - accuracy: 0.2492 - loss: 1.8759 - val_accuracy: 0.2640 - val_loss: 1.8056 - learning_rate: 3.0000e-04
Epoch 2/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 603s 2s/step - accuracy: 0.3794 - loss: 1.5432 - val_accuracy: 0.3743 - val_loss: 1.5588 - learning_rate: 3.0000e-04
Epoch 3/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 626s 2s/step - accuracy: 0.4537 - loss: 1.3796 - val_accuracy: 0.3962 - val_loss: 1.4985 - learning_rate: 3.0000e-04
Epoch 4/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 622s 2s/step - accuracy: 0.5071 - loss: 1.2619 - val_accuracy: 0.3657 - val_loss: 2.0505 - learning_rate: 3.0000e-04
Epoch 5/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 608s 2s/step - accuracy: 0.5413 - loss: 1.1901 - val_accuracy: 0.3066 - val_loss: 3.0092 - learning_rate: 3.0000e-04
Epoch 6/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 600s 2s/step - accuracy: 0.5627 - loss: 1.1320 - val_accuracy: 0.4943 - val_loss: 1.4297 - learning_rate: 3.0000e-04
Epoch 7/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 692s 2s/step - acc

Test Accuracy: 0.65
